In [22]:
from gensim.models import Word2Vec

import pandas as pd
from sklearn.externals import joblib
import re
from collections import defaultdict
import operator
import numpy as np

In [74]:
dfall = pd.read_csv("./NOTEEVENTS.csv")

/Users/tliu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [75]:
# Take first 10000 entries for training and next 10000 entries for testing
df1 = dfall[:10000]
df2 = dfall[10000:20000]

In [76]:
joblib.dump(df1,'data10.pkl')
joblib.dump(df2,'data10_2.pkl')

['data10_2.pkl']

In [77]:
df = joblib.load('data10.pkl')
df_test = joblib.load('data10_2.pkl')

In [78]:
df_text = df[['CHARTDATE','CATEGORY','DESCRIPTION','TEXT']]
df_text_test = df_test[['CHARTDATE','CATEGORY','DESCRIPTION','TEXT']]

In [65]:
from nltk.corpus import stopwords

sentences = []
stop_words = set(stopwords.words('english'))

for txt in df_text.TEXT:
    txt1 = re.sub('[\n]'," ",txt)
    txt1 = re.sub('[^A-Za-z ]+', '', txt1)
    mywords = txt1.split()
    # lowercase conversion
    mywords2 = [s.lower() for s in mywords]
    # stopword removal
    mywordsfinal = [w for w in mywords2 if not w in stop_words]
    sentences.append(mywordsfinal)

In [66]:
# train model
model = Word2Vec(sentences, min_count=1)

In [67]:
# summarize the loaded model
print(model)

Word2Vec(vocab=80296, size=100, alpha=0.025)


In [69]:
# access vector for one word
print(model['disease'])

[ 0.31556562  2.9643815   2.4030697  -1.786731   -4.189413    1.9433398
 -5.4507823   0.41236672 -7.0074086  -0.53422207  3.1250353  -0.20335048
 -2.574205    0.7145659  -2.066746    5.312218    1.9605073  -0.41838616
 -1.3179606  -2.084497    3.0499353   5.10113     2.2299852   1.4062076
 -0.13330488 -4.298199    3.1258724   0.18219504  1.0451035   3.1909857
  2.6948004   4.1382217  -3.6224606  -1.4307517   3.8671024   5.3570385
 -0.5258601  -2.3846426  -2.1641247  -0.61094147 -3.7927916   0.61409736
 -0.6978742  -0.17359234 -1.0963233  -2.595856    0.75404453 -0.6348068
 -1.9209205  -0.51159805  4.9157825  -0.5028961  -2.5326982  -1.3806129
 -0.40871972 -0.34002206  5.216468   -0.80550283 -0.6495808  -3.4492233
  4.2090993  -1.2606314  -1.0895176  -1.5606775   1.9389789  -3.0909638
 -0.1739206  -1.4980361   0.59233314 -0.4656584  -2.4303646   1.5182232
 -0.5131587   1.1930109  -1.184398    2.8290632  -1.3992509   2.2710426
 -0.79037434 -2.2007425  -1.3288292   0.6167452   0.5827304  

/Users/tliu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [70]:
# save model
model.save('model.bin')
# load model
new_model = Word2Vec.load('model.bin')
print(new_model)

Word2Vec(vocab=80296, size=100, alpha=0.025)


In [71]:
print(new_model.predict_output_word(['disease'], topn = 1))

[('infectious', 0.99999994)]


In [73]:
print (new_model.wv.most_similar(['diabetes'], topn = 10))

[('dependentdiabetes', 0.7202577590942383), ('dm', 0.6995397210121155), ('diabtes', 0.6465699672698975), ('dibetes', 0.634494960308075), ('prediabetes', 0.5752801895141602), ('honkdiabetes', 0.5701246857643127), ('mellitus', 0.5570030212402344), ('noninsulin', 0.5569166541099548), ('insulindependent', 0.5438898801803589), ('diabete', 0.5411747097969055)]


In [90]:
print ((df_text_test.TEXT)[10000])

Admission Date:  [**2116-11-16**]              Discharge Date:   [**2116-11-20**]

Date of Birth:  [**2035-12-10**]             Sex:   M

Service: MEDICINE

Allergies:
Sulfonamides / Iodine; Iodine Containing / Influenza Virus
Vaccine

Attending:[**First Name3 (LF) 6195**]
Chief Complaint:
Found down, hypotensive in the ED


Major Surgical or Invasive Procedure:
None


History of Present Illness:
80 yo man with history of neuroendocrine tumor s/p colostomy and
chemotherapy, paroximal atrial fibrillation on coumadin who
presents being found down by wife. [**Name (NI) **] had been feeling weak for
4 days. Then today, he was taking his sock off when he slipped
off the bed and fell to floor. His wife reports that he did not
hit his head.  He was unable to crawl to phone to call 911. His
wife arrived after approximately 3 hours and called EMS. BP
initially 140/90.
.
In the ED: VS 102.4, 150/88, HR 94, RR16, 98% RA, EKG no change
per ED (not avail). Labs: lactate 1.0, Creatinine, Hct at
base

In [89]:
print(new_model.predict_output_word(['prostate', 'status', 'post', 'definitive', 'radiation'], topn = 1))

[('cancer', 0.016078644), ('resection', 0.011121074), ('cholecystectomy', 0.0041043027), ('carcinoma', 0.0034370227), ('hysterectomy', 0.0031943342), ('xrt', 0.0019549986), ('colectomy', 0.0019386981), ('prostatectomy', 0.0017778605), ('excision', 0.0015646915), ('adenocarcinoma', 0.0013128865)]
